In [41]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
# reading the excel file
df_start = pd.read_excel('worldcities.xlsx')

#get a data frame with selected columns
FORMAT = ['city', 'lat', 'lng','country','iso2','population']
df_selected = df_start[FORMAT]

#insert in the dataframe the flag for the population
df_selected["flg_pop"]=df_selected.population.apply(lambda x: 1 if x > 200000 else 0 )

# sorting values by longitude
df_selected.sort_values(by=['lng'], inplace=True, ignore_index=True)

# searching the index of London (GB)
rslt_df = df_selected.loc[(df_selected['city'] == 'London') & 
              (df_selected['iso2']=='GB')] 
index_of_london = rslt_df.index[0]

# splitting the dataframe before and after the London index
df_a = df_selected.iloc[:index_of_london]
df_b = df_selected.iloc[index_of_london:]

# reshaping of dataframe
df = df_b.append(df_a).reset_index(drop=True)

<ipython-input-2-23e6180f0d65>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["flg_pop"]=df_selected.population.apply(lambda x: 1 if x > 200000 else 0 )
<ipython-input-2-23e6180f0d65>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.sort_values(by=['lng'], inplace=True, ignore_index=True)


In [52]:
class AroundTheWorld(object):
    
    def __init__(self,dataframe, city_start, country_start):
        self.dataframe=dataframe
        self.lat_max=None
        self.lat_min=None
        self.lng_min=None
        self.lng_max=None
        self.y_midpoint=None #change name(lat, lng city_step)
        self.city_start=city_start
        self.country_start=country_start
        self.city_step=city_start
        self.country_step=country_start
        
    def generate_grid(self, x_size=0.2, y_size=0.1):
        self.y_midpoint = tuple(self.dataframe.loc[(self.dataframe['city'] == self.city_step) & (self.dataframe['iso2']== self.country_step)][['lng','lat']].loc[0]) #midpoint of shortest side of rectangular
        #y_a=(y_midpoint[0],y_midpoint[1]-y_size/2)
        #y_b=(y_midpoint[0],y_midpoint[1]+y_size/2)
        #y_c=(y_b[0]+x_size,y_b[1])
        #y_d=(y_c[0],y_a[1])
        self.lat_max=self.y_midpoint[1]+y_size/2
        self.lat_min=self.y_midpoint[1]-y_size/2
        self.lng_min=self.y_midpoint[0]
        self.lng_max=self.y_midpoint[0]+x_size
        print(self.lat_max, self.lat_min, self.lng_min, self.lng_max)
        
    def query(self, n_min=3):
        grid_city=self.dataframe.loc[(self.dataframe['lat'] >= self.lat_min) &
                                     (self.dataframe['lat'] <= self.lat_max) & 
                                     (self.dataframe['lng'] <= self.lng_max) & 
                                     (self.dataframe['lng'] >= self.lng_min) & 
                                     (self.dataframe['city'] != self.city_step) ]
        print(grid_city)
        return grid_city
    
    def weight(self, grid_city):
        grid_city["distance"]=grid_city.apply(self.calculate_distance, axis=1, point_city_step=self.y_midpoint )
        grid_city.sort_values(by=['distance'], inplace=True)
        grid_city["weight"]=grid_city.apply(self.calculate_weigth, axis=1, country_step=self.country_step)
        print(grid_city)
        
    @staticmethod #decorator
    def calculate_weigth(row, country_step):
        pop_weigth=2 if row['flg_pop'] == 1 else 0
        country_weigth=2 if row['iso2'] != country_step else 0
        return pop_weigth + country_weigth
    
    @staticmethod
    def calculate_distance(row, point_city_step):
        return euclidean_distances([list(point_city_step)], [[row['lng'],row['lat']]])[0][0]
             

In [53]:
around = AroundTheWorld(df, "London", "GB")
around.generate_grid()
k=around.query()
around.weight(k)

51.557199999999995 51.4572 -0.1275 0.07250000000000001
            city      lat     lng         country iso2  population  flg_pop
5        Lambeth  51.4903 -0.1193  United Kingdom   GB      9675.0        0
6        Holborn  51.5172 -0.1182  United Kingdom   GB     13023.0        0
13       Brixton  51.4630 -0.1060  United Kingdom   GB     78536.0        0
14     Islington  51.5440 -0.1027  United Kingdom   GB    206125.0        1
18      Highbury  51.5520 -0.0970  United Kingdom   GB     26664.0        0
29  Spitalfields  51.5166 -0.0750  United Kingdom   GB     10286.0        0
40       Stepney  51.5152 -0.0462  United Kingdom   GB     16238.0        0
44       Hackney  51.5414 -0.0266  United Kingdom   GB     11734.0        0
61     Stratford  51.5423 -0.0026  United Kingdom   GB     51387.0        0
64      West Ham  51.5347  0.0077  United Kingdom   GB     15551.0        0
73     Kidbrooke  51.4650  0.0330  United Kingdom   GB     14300.0        0
80      East Ham  51.5323  0.0554

<ipython-input-52-f742d0d93852>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grid_city["distance"]=grid_city.apply(self.calculate_distance, axis=1, point_city_step=self.y_midpoint )
<ipython-input-52-f742d0d93852>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grid_city.sort_values(by=['distance'], inplace=True)
<ipython-input-52-f742d0d93852>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

In [20]:
plt.figure(figsize=(200,100))
plt.scatter(df.lng,df.lat, color="green", marker="o", alpha=0.5)
plt.scatter(k.lng,k.lat,color="red", marker="+")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid()
plt.show()

In [ ]:
df.loc[(df['lat'] >= 51.507) & (df['lat'] <= 52.507) & (df['lng'] <= -0.1) & (df['lng'] >= -0.2)]